In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image
import os

In [ ]:
path_df = pd.read_csv('datasets/thyroid_ultrasonic_image/batch1_image/batch1_image.csv')
image_label = pd.read_csv('datasets/thyroid_ultrasonic_image/batch1_image/batch1_image_label.csv')
# Merge on patient name
merged_df = path_df.merge(image_label, on='patient_name', how='inner')

# Final structure
final_df = merged_df[['path', 'patient_name', 'histo_label']]
final_df = final_df.rename(columns={'path': 'image_path', 'histo_label': 'label'})
final_df['image_path'] = 'datasets/thyroid_ultrasonic_image/batch1_image/dataset/' + final_df['image_path']
print(final_df.head())

In [12]:
cnn_model = EfficientNetB0(include_top=False, pooling='avg', weights='imagenet', input_shape=(224, 224, 3))
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    return img_array  

In [13]:
from tqdm import tqdm
def extract_patient_features(df):
    patient_features = []
    patient_labels = []

    grouped = df.groupby('patient_name')
    for patient, group in tqdm(grouped, desc="Extracting patient features"):
        images = group['image_path'].tolist()
        label = group['label'].iloc[0]

        image_vecs = []
        for img_path in images:
            if not os.path.exists(img_path):
                print(f"[SKIPPED] Missing file: {img_path}")
                continue  # skip if file doesn't exist

            try:
                img = preprocess_image(img_path)
                feat = cnn_model.predict(np.expand_dims(img, axis=0), verbose=0)
                image_vecs.append(feat[0])
            except Exception as e:
                print(f"[ERROR] Problem with {img_path}: {e}")
                continue

        if len(image_vecs) == 0:
            print(f"[WARNING] No valid images for patient: {patient}, skipping")
            continue

        # Aggregate all image vectors into one patient vector (mean pooling)
        patient_vector = np.mean(image_vecs, axis=0)
        patient_features.append(patient_vector)
        patient_labels.append(label)

    return np.array(patient_features), np.array(patient_labels)

In [ ]:
X_train, y_train = extract_patient_features(final_df)
np.save('X_train.npy', X_train)
np.save('y_train.npy', y_train)

Extracting patient features:   0%|          | 0/601 [00:00<?, ?it/s]

[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/0_002.Jpg


Extracting patient features:   0%|          | 3/601 [00:03<12:20,  1.24s/it]

[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/2_008.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/2_009.Jpg


Extracting patient features:   1%|          | 4/601 [00:04<11:54,  1.20s/it]

...

Extracting patient features: 100%|█████████▉| 598/601 [10:50<00:02,  1.08it/s]

[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/597_006.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/597_007.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/597_008.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/597_009.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/597_010.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/597_011.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/597_012.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/597_013.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/597_014.Jpg


Extracting patient features: 100%|█████████▉| 599/601 [10:51<00:02,  1.00s/it]

[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/598_010.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/598_011.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/598_012.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/598_013.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/598_014.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/599_004.Jpg


Extracting patient features: 100%|██████████| 601/601 [10:54<00:00,  1.09s/it]


In [4]:
X_train = np.load('X_patient_features.npy')
y_train = np.load('y_patient_labels.npy')

In [9]:
test_path_df = pd.read_csv('datasets/thyroid_ultrasonic_image/batch2_image/batch2_image.csv')
image_label_test = pd.read_csv('datasets/thyroid_ultrasonic_image/batch2_image/batch2_image_label.csv')
# Merge on patient name
merged_df = test_path_df.merge(image_label_test, on='patient_name', how='inner')

# Final structure
test_df = merged_df[['path', 'patient_name', 'histo_label']]
test_df = test_df.rename(columns={'path': 'image_path', 'histo_label': 'label'})
test_df['image_path'] = 'datasets/thyroid_ultrasonic_image/batch1_image/dataset/' + test_df['image_path']
print(test_df.head())

                                          image_path  patient_name  label
0  datasets/thyroid_ultrasonic_image/batch1_image...           195      1
1  datasets/thyroid_ultrasonic_image/batch1_image...           195      1
2  datasets/thyroid_ultrasonic_image/batch1_image...           195      1
3  datasets/thyroid_ultrasonic_image/batch1_image...           195      1
4  datasets/thyroid_ultrasonic_image/batch1_image...           195      1


In [14]:
X_test, y_test = extract_patient_features(test_df)
np.save('X_test.npy', X_test)
np.save('y_test.npy', y_test)

Extracting patient features:   3%|▎         | 8/241 [00:00<00:03, 75.57it/s]

[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/0_001_174555.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/0_002_174555.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/0_003_174555.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/0_004_174555.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/0_005_174556.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/0_006_174556.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/0_007_174556.Jpg
[WARNING] No valid images for patient: 0, skipping
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/1_001_135206.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/1_002_135206.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/1_003_

Extracting patient features:  12%|█▏        | 28/241 [00:00<00:02, 85.70it/s]

[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/18_017_112709.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/18_018_112709.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/18_019_112709.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/18_020_112709.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/18_021_112709.Jpg
[WARNING] No valid images for patient: 18, skipping
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/19_001_180857.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/19_002_180858.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/19_003_180858.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/19_004_180858.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/data

Extracting patient features:  20%|██        | 49/241 [00:00<00:02, 86.68it/s]

...

[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/202_006_103831.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/202_007_103832.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/202_008_103832.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/202_009_103832.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/202_010_103832.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/202_011_103833.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/202_012_103833.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/202_013_103833.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/202_014_103834.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/202_015_103834.Jpg
[SKIPPED] Missing fi

Extracting patient features:  92%|█████████▏| 221/241 [00:02<00:00, 78.21it/s]

[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/213_001_181522.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/213_002_181523.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/213_003_181523.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/213_004_181523.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/213_005_181524.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/213_006_181524.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/213_007_181525.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/213_008_181525.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/213_009_181525.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/213_010_181526.Jpg
[SKIPPED] Missing fi

Extracting patient features: 100%|██████████| 241/241 [00:02<00:00, 87.12it/s]

[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/230_004_174428.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/230_005_174428.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/230_006_174428.Jpg
[WARNING] No valid images for patient: 230, skipping
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/231_001_175541.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/231_002_175541.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/231_003_175542.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/231_004_175542.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/231_005_175542.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_image/dataset/231_006_175543.Jpg
[SKIPPED] Missing file: datasets/thyroid_ultrasonic_image/batch1_